<a href="https://github.com/shalvamist/GPTme/blob/main/Examples/notebooks/GPTme_CRAG_terminal_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Ollama + Corrective RAG in Colab

### This example will show how to use GPTme package to run a corrective RAG using LangGraph.

###### Credit to Langchain Blog for the CRAG logic - https://blog.langchain.dev/agentic-rag-with-langgraph/

###### Credit to Gruff and his post on StackOverflow for making Ollama run in colab -
https://stackoverflow.com/questions/77697302/how-to-run-ollama-in-google-colab



# Setting up GPTme

In [3]:
%%bash
git clone https://github.com/shalvamist/GPTme.git
cd GPTme/
source setup.sh

  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune

fatal: destination path 'GPTme' already exists and is not an empty directory.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.24.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.0 which is incompatible.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.0 which is incompatible.
fatal: destination path 'llama.cpp' already exists and is not an empty directory.
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached scikit_build_core-0.8.2-py3-none-any.whl (140 kB)
    Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
    Using cach

# Installing Ngrok & Ollama

#### In order to run Ollama in Colab you will need a Ngrok key - it's free (for now)
#### Sign up here and get your dev key - https://ngrok.com/
#### Fill in the key in the second cell 'token=...'

In [1]:
!pip install pyngrok
!sudo apt-get install lshw
# Download and run the Ollama Linux install script
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
import threading
import time
import os
import asyncio
from pyngrok import ngrok
import threading
import queue
import time
from threading import Thread

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token="YOUR KEY GORS HERE"
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [3]:
# Create a queue to share data between threads
url_queue = queue.Queue()

# Start ngrok in a separate thread
ngrok_thread = StoppableThread(target=start_ngrok, args=(url_queue, StoppableThread.is_stopped))
ngrok_thread.start()

# Wait for the ngrok tunnel to be established
while True:
    try:
        public_url = url_queue.get()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

Error in start_ngrok: 'function' object has no attribute 'is_set'Ngrok tunnel established at: NgrokTunnel: "https://ba0e-34-32-173-22.ngrok-free.app" -> "http://localhost:11434"



In [4]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
# os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [5]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve


# Pulling Ollama Mistral - This might take a few min

#### you can pull any model you want available from Ollama - https://ollama.com/library

#### I recommend to wait 5 min till the model download & verification is done

In [12]:
async def ollama_pull(model="mistral:instruct"):
    await run_process(['ollama', 'pull', model])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, ollama_pull()))
thread.start()

>>> starting ollama pull mistral:instruct


In [7]:
!ollama pull mistal:instruct

[GIN] 2024/03/15 - 23:12:46 | 200 |      35.928µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ [GIN] 2024/03/15 - 23:12:47 | 200 |  1.290638279s |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
Error: pull model manifest: file does not exist


# Corrective RAG (CRAG) app

#### This is a simple CRAG graph, you can find the whole pipe under GPTme/pipes/loacl_crag.py
#### There are two main retrievers (MMR, Similarity) that are being used to generate context, the LLM will generate a response only if it deciedes it has enough relevant context

In [8]:
!python GPTme/Examples/crag_example.py

Hello I am here to help you find answers from your documentation.
Please let me know what you would like to know. to end our conversation please enter 'bye'
User entry - hello
---RETRIEVE---
Importing: riscv-spec-v2.2.pdf
/content/GPTme/GPTme/../SOURCE_DOCS/riscv-spec-v2.2.pdf loaded.

Building DB from sources - using embedding model BAAI/bge-base-en-v1.5
"Node 'mmr_retrieve':"
'\n---\n'
---CHECK RELEVANCE---
Using Langchin to mount mistral:instruct with Ollama
time=2024-03-15T23:13:47.933Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-03-15T23:13:47.934Z level=INFO source=gpu.go:119 msg="CUDA Compute Capability detected: 7.5"
time=2024-03-15T23:13:47.934Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-03-15T23:13:47.934Z level=INFO source=gpu.go:119 msg="CUDA Compute Capability detected: 7.5"
time=2024-03-15T23:13:47.934Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-03-15T23:13:47.951Z level=INFO source=dyn_ext_server.go:90 msg="Lo